# Exercise 5: Camera Calibration (from Existing Images)

In this exercise you will:
- Learn how to calibrate your camera from images already taken by a camera.

To calibrate your own camera (i.e. by taking pictures, or a set of video frames, yourself), the OpenCV tutorials walk you through the process. The tutorials can be found using the links below (and also include tutorials for 3D reconstruction, once you have calibrated your camera).

https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_table_of_contents_calib3d/py_table_of_contents_calib3d.html

https://docs.opencv.org/master/d9/db7/tutorial_py_table_of_contents_calib3d.html

Camera calibration can take some time. Therefore, in this exercise, we will stop the calibration before the calibrated camera parameters are saved to the file system (so we don't have to wait 30 minutes to an hour to calibrate our cameras). The folder "camera_params" already contains the calibration parameters from a previous run, so we can use these for demonstrating 3D reconstruction.

In the next exercise, we will use the camera calibration parameters from the folder "camera_params" to reconstruct a 3D scene from two 2D images.

The OpenCV function `ret, corners = cv.findChessboardCorners(gray_image, chessboard_size, None)` takes in a `gray_image` (or video frame) of a chess board and a `chessboard_size` (the real width and height of a square in cm on the chess board), and returns the detected `corners`, and the return value `ret`, which will be `True` if the `corners` where properly detected, otherwise `False`.

A chess board image is ideal for camera calibration, since it's easy to find the corners (e.g. using Harris Corner Detection). That's what the `cv.findChessboardCorners()` function does.

Once we have found the `corners`, we can use the OpenCV function `cv.cornerSubPix(gray_image, corners, (5,5), (-1,-1), criteria)` to slide a 5x5 window `(5,5)` over the `gray_image` to refine the positions of the `corners` to sub-pixel accuracy. The `(-1,-1)` parameter is unimportant, and the `criteria` parameter determines how the search for sub-pixels is done (how many iterations, etc.).

Then we use the OpenCV function `ret, K, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray_image.shape[::-1], None, None)` to get the required camera calibration parameters. We also need to find the camera's focal length. In this example, we will use the `PIL` python package to do this.

Finally, we store the camera calibration parameters to the file system.

As a first step, let's import the python modules we need.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import glob
import PIL.ExifTags
import PIL.Image

## Camera Calibration



In [2]:
#============================================
# Camera calibration
#============================================

# Define chess board size.
chessboard_size = (7,5)

# Define arrays to save detected points
obj_points = [] # 3D points in real world space 
img_points = [] # 3D points in image plane

# Prepare grid of 3D points on the real chess board
# (the Z-coordinate is assumed to be 0, i.e. we only
# shift the chess board along the X and Y axes when
# taking pictures from different view points).
objp = np.zeros((np.prod(chessboard_size),3), dtype=np.float32)
objp[:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1,2)

# Read in all calibration images
calibration_paths = glob.glob('./calibration_images/*')

# Iterate over the images to find the intrinsic camera matrix
for image_path in calibration_paths:

	# Load image and convert to gray scale
	image = cv.imread(image_path)
	gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
	
	print("Image loaded, finding chess board corners ...")
	
	# Find chessboard corners
	ret, corners = cv.findChessboardCorners(gray_image, chessboard_size, None)

	# If the chess board corners were detected
	if ret == True:
		print(f'Chessboard detected in image: {image_path}')

		# Add 3D points on the real chess board
		obj_points.append(objp)
		
		# Define criteria for subpixel accuracy
		criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
		
		# Refine corner location (to subpixel accuracy) based on criteria
		cv.cornerSubPix(gray_image, corners, (5,5), (-1,-1), criteria)
		
		# Add detected corners in the image to the list of image points
		img_points.append(corners)

# Calibrate camera
ret, K, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray_image.shape[::-1], None, None)

# Save parameters into numpy file
np.save("./camera_params/ret", ret)
np.save("./camera_params/K", K)
np.save("./camera_params/dist", dist)
np.save("./camera_params/rvecs", rvecs)
np.save("./camera_params/tvecs", tvecs)

# Get "exif" data in order to get focal length 
exif_img = PIL.Image.open(calibration_paths[0])

exif_data = {
	PIL.ExifTags.TAGS[k]:v
	for k, v in exif_img._getexif().items()
	if k in PIL.ExifTags.TAGS}

# Get focal length
focal_length = exif_data['FocalLength']

# Save focal length
np.save("./camera_params/FocalLength", focal_length)

# Calculate projection error
mean_error = 0
for i in range(len(obj_points)):
	img_points2, _ = cv.projectPoints(obj_points[i], rvecs[i], tvecs[i], K, dist)
	error = cv.norm(img_points[i], img_points2, cv.NORM_L2)/len(img_points2)
	mean_error += error

# Print the total projection error
total_error = mean_error/len(obj_points)
print(f'total_error: {total_error}')

Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7769.JPG
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7770.JPG
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7771.JPG
Image loaded, finding chess board corners ...
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7773.JPG
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7774.JPG
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7775.JPG
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_images\IMG_7776.JPG
Image loaded, finding chess board corners ...
Image loaded, finding chess board corners ...
Image loaded, finding chess board corners ...
Chessboard detected in image: ./calibration_im